In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Arial;  font-size:10pt; line-height:140%; }
div.text_cell_render.rendered_html{font-size:10pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:11pt;padding:4px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="5" color="red">ch14.웹데이터 수집</font></b>
# 1절. BeautifuSoup과 parser
```pip install bs4``` 아나콘다를 설치하면 7500여개의 패키지 설치
- 공식 사이트 : https://www.crummy.com/software/BeautifulSoup/
- Documentation : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [2]:
import requests # HTTP 요청 처리 lib
from requests_file import FileAdapter

In [18]:
s = requests.Session()  # HTTP 요청관리를 위한 세션 객체
s.mount("file://",FileAdapter())
response = s.get("file:///AI_X/source/ch01_python/data/ch14_sample.html")
response

<Response [200]>

In [4]:
response.status_code  
# 200 : 정상
# 404 : 없는 페이지
# 406 : get, post 오류
# 500 : 파이썬 문법 에러

200

In [7]:
response.content # 바이너리 형식의 내용


b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [8]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [9]:
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [19]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,   # response.text
                    'html.parser')
# soup

In [24]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처름 나오는 h1태그 하나만
print('el : ', el)
print('el.text :', el.text)
print('el.text :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 title 속성 :', el.attrs['title']) # 철자가 틀리거나 없으면 에러
print('el의 title 속성 :', el.attrs.get('title')) # 철자가 틀리거나 없으면 None 을 출력
print('el의 이름 :', el.name)

el :  <h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>
el.text : Hello, CSS
el.text : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text', 'title': 'hi'}
el의 title 속성 : hi
el의 title 속성 : hi
el의 이름 : h1


In [30]:
# suop.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로
el = soup.select('h1')
print('리스트 el :', el)
print('el의 text들 :', [e.text for e in el])
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class속성들 :', [e.attrs.get('class') for e in el])

리스트 el : [<h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el의 text들 : ['Hello, CSS', 'Hi, CSS']
el의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text', 'title': 'hi'}, {'class': ['css']}]
el의 class속성들 : [['greeting', 'css'], ['css']]


In [37]:
# select_one(선택자)와 find(태그 , 속성)
print('select_one :', soup.select_one('h1.css'))
print('find : ', soup.find('h1',{'class':'css'}))
print('find :', soup.find('h1', class_='css'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1',{'id':'text'}))
print()
print('select')

select_one : <h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>
find :  <h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>

select


In [39]:
# select(선택자)와 find_all(태그, 속성)
print('모든 h1.css, span 태그(select) :', soup.select('h1.css, span'))
print('모근 h1.css, span태그(find_all) :', soup.find_all(['h1','span'], class_='css'))

모든 h1.css, span 태그(select) : [<h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모근 h1.css, span태그(find_all) : [<h1 class="greeting css" id="text" title="hi">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [46]:
# 없는 엘리먼트 찾기
print('find(빈list) :', soup.find_all('a', class_='css'))
print('find(None) : ', soup.find('a', class_='css'))
print('select(빈list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))

find(빈list) : []
find(None) :  None
select(빈list) : []
select_one(None) : None


# 2절. 정적 웹 데이터 수집(정적 웰크롤링)
- json, html, xml
## 2.1 JSON파일
- request 모듈(get)
- uellib.request 모듈(uelopne)

In [ ]:
# 크롤링 허용 범위는 사이트마다 ~/robots.txt에서 확인할 수 있습니다

In [112]:
# 방법 1
import requests
requests = requests.get('http://api.github.com')
requests, requests.status_code

(<Response [200]>, 200)

In [114]:
# 방법 2
from urllib.request import urlopen
# response = urlopen('http://api.github.com')
# response

In [123]:
text = '{"속성1" :"값1","속성2" :"값2"}'

In [124]:
# 문자(딕셔너이 타입)를 딕셔너리
# '{'속성1': '값1','속성2' : '값2'}' => {'속성1': '값1','속성2' : '값2'}
import json
json.loads(text)

{'속성1': '값1', '속성2': '값2'}

## 2.2 html 파일
### 1) 환율정보 가져오기(네이버->증권=>시장지표)
- https://finance.naver.com/marketindex/

In [56]:
# 방법 1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response, response.status_code
# response.content
# response.content.decode('cp949') == response.text
soup = BeautifulSoup(response.text,'html.parser')


In [67]:
# 방법 2
from urllib.request import urlopen
response = urlopen(url)
# response.status 상태코드
# response.read().decode('cp949')  # == response.content
soup = BeautifulSoup(response,  # response.read()
                     'html.parser')


In [69]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [81]:
price = soup.select('div.head_info> span.value')
[p.text for p in price]
[round(float(p.text.replace(',',''))) for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

[1375, 953, 1556, 191, 144, 1, 1, 99, 61, 1633, 3300, 146121]

In [76]:
out = '1,375.20'
round(float(''.join(out.split(','))))


1375

In [78]:
round(float(out.replace(',','')))

1375

In [84]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7,'') # 7번째 index에 ''추가
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [89]:
status = soup.select('div.head_info > span.blind')
[t.text for t in title]
[p.text for p in price]
unit
[s.string for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [91]:
len(title), len(price) , len(unit), len(status)

(12, 12, 12, 12)

In [97]:
for idx in range(len(title)):
    print('{}. {}: {}{} {}'.format(idx,
                                title[idx].text,
                                price[idx].text,
                                unit[idx],
                                status[idx].text))

0. 미국 USD: 1,375.20원 하락
1. 일본 JPY(100엔): 952.72원 하락
2. 유럽연합 EUR: 1,555.56원 하락
3. 중국 CNY: 191.08원 하락
4. 달러/일본 엔: 144.3800엔 상승
5. 유로/달러: 1.1342달러 하락
6. 영국 파운드/달러: 1.3520달러 하락
7. 달러인덱스: 99.4200 상승
8. WTI: 60.89달러 하락
9. 휘발유: 1633.17원 하락
10. 국제 금: 3300.4달러 하락
11. 국내 금: 146120.87원 상승


In [107]:
for t, p, u, s in zip(title, price, unit, status):
    print("{} : {}{} {}".format(t.text,
                               p.text,
                               u,
                               p.text))

미국 USD : 1,375.20원 1,375.20
일본 JPY(100엔) : 952.72원 952.72
유럽연합 EUR : 1,555.56원 1,555.56
중국 CNY : 191.08원 191.08
달러/일본 엔 : 144.3800엔 144.3800
유로/달러 : 1.1342달러 1.1342
영국 파운드/달러 : 1.3520달러 1.3520
달러인덱스 : 99.4200 99.4200
WTI : 60.89달러 60.89
휘발유 : 1633.17원 1633.17
국제 금 : 3300.4달러 3300.4
국내 금 : 146120.87원 146120.87


In [111]:
for idx, (t, p, u, s) in enumerate(zip(title, price, unit, status)):
    print("{}. {} : {}{} {}".format(idx+1,
                                t.text,
                               p.text,
                               u,
                               p.text))

1. 미국 USD : 1,375.20원 1,375.20
2. 일본 JPY(100엔) : 952.72원 952.72
3. 유럽연합 EUR : 1,555.56원 1,555.56
4. 중국 CNY : 191.08원 191.08
5. 달러/일본 엔 : 144.3800엔 144.3800
6. 유로/달러 : 1.1342달러 1.1342
7. 영국 파운드/달러 : 1.3520달러 1.3520
8. 달러인덱스 : 99.4200 99.4200
9. WTI : 60.89달러 60.89
10. 휘발유 : 1633.17원 1633.17
11. 국제 금 : 3300.4달러 3300.4
12. 국내 금 : 146120.87원 146120.87


### 2) 이번주 로또번호 출력
- https://www.dhlottery.co.kr/gameResult.do?method=byWin(google) "로또번호 당첨번호 검색)
```
    1173회(2025년 05월 24일 추첨)
    당첨번호[1 5 18 20 30 35]
    보너스 3
```

In [181]:
import requests
from bs4 import BeautifulSoup
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')


In [182]:
# 방법2의 sou객체 생성
from urllib.request import urlopen
response = urlopen(url)
soup = BeautifulSoup(response, "html.parser")

In [159]:
times = soup.select_one('div.win_result > h4 > strong').text
date = soup.select_one('div.win_result >p.desc').text
print(times, date)
title = soup.select_one('div.num.win > strong').text
lotto_number = soup.select('div.num.win > p > span')
print(title, [int(lotto.text) for lotto in lotto_number])
title2 = soup.select_one('div.num.bonus > strong').string
bonus = soup.select_one('div.num.bonus > p > span').text
print(title, bonus)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
당첨번호 3


In [183]:
win_result = soup.find('div', class_="win_result")
times = win_result.find('strong').text
date = soup.find('p', class_='desc').text
print(times, date)
num_win = soup.find('div', class_=['win'])
title1 = num_win.find('strong').text
lotto = num_win.find_all('span')
print(title1, [int(lo.text) for lo in lotto])
num_bonus = soup.find('div', class_=['bonus'])
title2 = num_bonus.find('strong').text
bonus_number = num_bonus.find('span').text
print(title2, bonus_number)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 3


### 3) 다음 검색 리스트
```
no   title  link
0 [비트코인 2025] 백아관 크립토 차르 ~ https://v.daum.net/v/20250528103907230
1 [비즈 나우] 비트코인 2035 컨퍼런 ~ https://v.daum.net/v/20250528075215864
```

In [189]:
# 방법 1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8' + word
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
items_fied_list = []  # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_fied_list.append({'no':idx+1,
                           'title':item.text,
                           'link':item.attrs['href']})
import pandas as pd
pd.DataFrame(items_fied_list)

,no,title,link
0,1,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,2,EU관세 발언 후 10만8000달러선 회복 비트코인…파키스탄 ‘비트코인 채굴’ 전...,http://v.daum.net/v/20250526080440640
2,3,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
3,4,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
4,5,"""비트코인 산다""…트럼프家, 25억달러 자금 조달 추진[코인브리핑]",http://v.daum.net/v/20250528111837527
5,6,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
6,7,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
7,8,"10만 달러, 11만 달러… 트럼프 관세 변덕에 비트코인 '출렁'",http://v.daum.net/v/20250527132843430
8,9,'해킹 산업화'의 원흉 '비트코인'…위치추적 안 되자 활개[은폐⑥],http://v.daum.net/v/20250527090029503
9,10,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853


In [207]:
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q='+word
response = requests.get(url)
soup = BeautifulSoup(response.text, # response.content
                    "html.parser")
items_find_list = [] # 검색한 결과를 담을 2차원 리스트
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])
# items_find_list
import pandas as pd
pd.DataFrame(items_find_list, columns=['no', 'title', 'link'])

,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
2,2,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
3,3,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
4,4,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
5,5,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
8,8,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [220]:
# 다음 뉴스 검색(키워드, 원하는 페이지수)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
keyword = '비트코인'
page = 2
#url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&q={keyword}&p={page}'
url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
params = {'q':keyword, 'p':page }
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text, 'html.parser')
items_fied_list = []  # 검색한 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_fied_list.append({'no':idx+1,
                           'title':item.text,
                           'link':item.attrs['href']})
    print({'no': (page-1)*10+idx,
            'title':item.text,
            'link':item.attrs['href']})

{'no': 10, 'title': ' 트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고 ', 'link': 'http://v.daum.net/v/20250528084245020'}
{'no': 11, 'title': ' 연일 최고가 경신하는 비트코인…투자자들 강 건너 불구경하는 이유는 ', 'link': 'http://v.daum.net/v/20250526212400275'}
{'no': 12, 'title': ' "비트코인 산다"…트럼프家, 25억달러 자금 조달 추진[코인브리핑] ', 'link': 'http://v.daum.net/v/20250528111837527'}
{'no': 13, 'title': ' [김대호 박사의 오늘의 키워드] 소비자신뢰지수·비트코인 2025·머스크 한마디·애플 괘씸죄 ', 'link': 'http://v.daum.net/v/20250528071325361'}
{'no': 14, 'title': ' 잘 나가는 비트코인 산다는데 왜?…트럼프 미디어 주가 10% 급락 [투자360] ', 'link': 'http://v.daum.net/v/20250528074002668'}
{'no': 15, 'title': ' "테슬라 팔고 갈아탔어요"…자녀계좌 보던 엄마들 \'돌변\' [마켓PRO] ', 'link': 'http://v.daum.net/v/20250528094602647'}
{'no': 16, 'title': ' 스트래티지, 비트코인 4020개 추가 매입...보유량 58만개 돌파 ', 'link': 'http://v.daum.net/v/20250527081116129'}
{'no': 17, 'title': ' 달러 스테이블코인, 국내 거래 확산…당국 외환흐름 통제력 약화 우려 ', 'link': 'http://v.daum.net/v/20250528152520516'}
{'no': 18, 'title': ' 비트코인과 엇갈리는 이더리움·리플, 왜? ', 'link': 'http://v.daum.net/v/20

In [252]:
# 다음 뉴스 검색(키워드, 원하는 페이지수)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 reuen'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page }
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    items_fied_list = []  # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_fied_list.append({'no':idx+1,
                           'title':item.text,
                           'link':item.attrs['href']})
    return items_fied_list

In [230]:
collect_list('대선', 2)

[{'no': 1,
  'title': ' [내일 사전투표] 대선 사전투표율과 본투표율 상관성↑...당락 결정타 되나 ',
  'link': 'http://v.daum.net/v/20250528152617559'},
 {'no': 2,
  'title': ' 기자와 술자리 졸던 이준석…“없애버려야!” 눈 번쩍 뜬 주제 [대선주자 탐구] ',
  'link': 'http://v.daum.net/v/20250528152233378'},
 {'no': 3,
  'title': ' [대선 D-6] 대선 후보들, 사전투표 장소도 신경전 ',
  'link': 'http://v.daum.net/v/20250528153517087'},
 {'no': 4,
  'title': ' 이재명 대선 공약집 발표…검찰·사법 개혁하고 기후에너지부 신설 ',
  'link': 'http://v.daum.net/v/20250528160628551'},
 {'no': 5,
  'title': " '이준석 대선후보 사퇴 요구 대학생 긴급 기자회견' ",
  'link': 'http://v.daum.net/v/20250528155311862'},
 {'no': 6,
  'title': ' 부산 다시 찾은 김문수, ‘사전 투표’ 재차 독려 [21대 대선] ',
  'link': 'http://v.daum.net/v/20250528155306855'},
 {'no': 7,
  'title': " [살까팔까] '지배구조 개선' 대선 공약에…한화·효성 등 지주사株 '훨훨' ",
  'link': 'http://v.daum.net/v/20250528155002701'},
 {'no': 8,
  'title': ' “일요일 마트 열까 말까”…대선에 따른 대형마트 의무 휴업 향방은 ',
  'link': 'http://v.daum.net/v/20250528160004191'},
 {'no': 9,
  'title': ' "尹 아바타", "이준석 사퇴해야"…마지막 대선 토론 평가는?[노컷투표] ',
  'l

In [253]:
# 원하는 keyword로 다음검색
result = []
keyword = "청바지"
pages = 4
for page in range(1, pages+1):
    result.extend(collect_list(keyword, page))
    time.sleep(1)
result_df = pd.DataFrame(result)
result_df.head()

""


In [260]:
keywords = ['청바지','면바지']
pages = 3
result0 = [] # 청바지 검색결과 50개
result1 = [] # 면바지 검색결과 50개
for i, keyword in enumerate(keywords):
    for page in range(1, pages+1):
        print(f'~ ~ ~{i}번째 {keyword} {page}검색 중입니다 ~ ~ ~')
        if i==0:
            result0.extend(collect_list(keyword, page))
        else:
            result1.extend(collect_list(keyword, page))
        time.sleep(3)

~ ~ ~0번째 청바지 1검색 중입니다 ~ ~ ~
~ ~ ~0번째 청바지 2검색 중입니다 ~ ~ ~
~ ~ ~0번째 청바지 3검색 중입니다 ~ ~ ~
~ ~ ~1번째 면바지 1검색 중입니다 ~ ~ ~
~ ~ ~1번째 면바지 2검색 중입니다 ~ ~ ~
~ ~ ~1번째 면바지 3검색 중입니다 ~ ~ ~


In [261]:
result0_df = pd.DataFrame(result0)
result0_df.head()

,no,title,link
0,1,"'이병헌♥' 이민정, 매장 통째로 빌려 청바지 쇼핑",http://v.daum.net/v/20250528050507783
1,2,"“추앙받을 만 하네”… 김지원, 셔츠에 청바지만 입어도 공항 ‘정지’",http://v.daum.net/v/20250526182411395
2,3,흰 티셔츠에 청바지 정석대로 입는 법,http://v.daum.net/v/20250527185425864
3,4,"이민정, 매장 빌려 폭풍 쇼핑 ""10년 전 샀던 청바지는 응급실""(MJ)",http://v.daum.net/v/20250527191015309
4,5,"‘이병헌♥’ 이민정, 청바지 사는데 매장 통째로 빌렸다..남다른 쇼핑 클래스 “나...",http://v.daum.net/v/20250527192310643


In [262]:
result1_df = pd.DataFrame(result1)
result1_df.head()

,no,title,link
0,1,사천에서 쾌청한 날씨 속 탁 트인 남해 풍경 즐겨요!,http://v.daum.net/v/20250528104429529
1,2,"윤승아, ♥김무열 쏙 빼닮은 깜찍 子 하객룩 공개‥벌써 우월한 유전자",http://v.daum.net/v/20250526145941690
2,3,"이번 주 포천 여행,반팔 하나로 충분할까?",http://v.daum.net/v/20250527171732518
3,4,"이재명, 첫 대학 방문…이준석 겨냥?",http://v.daum.net/v/20250526191942751
4,5,"""나는 젊어"" 팔소매 접은 이준석의 '흰셔츠' 속에도 뼈가 담겼다[21대 대선 리...",http://v.daum.net/v/20250525070013619


In [263]:
result0_df.to_csv('data/ch14_'+keywords[0]+ '.csv', index=False, encoding='cp949')
result1_df.to_csv('data/ch14_'+keywords[1]+ '.csv', index=False)

### 4) user-agent를 추가하여 크롤링
- urlopne()함수를 사용하면 크롤링이 안 되는 사이트
- user-agent를 추가하여 크롤링

In [258]:
# 방법 2
from urllib.request import urlopen, Request
import urllib.parse
word = '비트코인'
word = urllib.parse.quote(word)
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8' + word
# User-Agent 를 추가하여 브라우저처럼 보이게 포장
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
response = urlopen(request)
soup = BeautifulSoup(response, "html.parser")
def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 reuen'
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page }
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    items_fied_list = []  # 검색한 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_fied_list.append({'no':idx+1,
                           'title':item.text,
                           'link':item.attrs['href']})
    return items_fied_list

In [259]:
collect_list('청바지', 2)

[{'no': 1,
  'title': ' 60대인데요, 청바지를 다시 입었습니다 ',
  'link': 'http://v.daum.net/v/20250430145405823'},
 {'no': 2,
  'title': " '45kg' 윤은혜, 청바지가 헐렁…'40세' 비현실적 자기관리 끝판왕 ",
  'link': 'http://v.daum.net/v/20250513174826265'},
 {'no': 3,
  'title': ' 173㎝ 장원영 맵시 美쳤네, 청바지 하나 입었을 뿐인데 [스타화보] ',
  'link': 'http://v.daum.net/v/20250522115656470'},
 {'no': 4,
  'title': ' 손담비, 출산 한 달도 안 됐는데 외출…청바지 폼 애엄마 맞나 ',
  'link': 'http://v.daum.net/v/20250506171902866'},
 {'no': 5,
  'title': ' 48세 김희선, 흰 셔츠+청바지 기본템 입었을 뿐인데‥미모 레전드 ',
  'link': 'http://v.daum.net/v/20250511134648392'},
 {'no': 6,
  'title': ' 윤은혜, 40세 맞아?…청바지도 헐렁한 극세사 몸매+동안 미모 ',
  'link': 'http://v.daum.net/v/20250515063005066'},
 {'no': 7,
  'title': ' “브래지어에 청바지만”…20년만에 속옷 모델 된 추성훈 아내 야노 시호 ‘파격’ ',
  'link': 'http://v.daum.net/v/20250515195400237'},
 {'no': 8,
  'title': ' ‘청바지’ 입은 안철수 ‘학식’ 이준석 쫓아갔지만 ',
  'link': 'http://v.daum.net/v/20250521160401791'},
 {'no': 9,
  'title': " 길은혜,'청바지 핏 뽐내며 입장' [사진] ",
  'link': 'http://v.daum.net/v/20250

- User-Agent를 사용하여야 하는 경우 : https://www.melon.com/chart/

In [266]:
# 방법 1
import requests
from bs4 import BeautifulSoup
url = 'https://www.melon.com/chart/'
melonpage = requests.get(url)
melonpage.status_code

406

In [269]:
# 방법 2
from urllib.request import urlopen, Request
#melonpage = urlopen(url)

In [290]:
# User-Agent추가
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'}
# 방법 2
request = Request(url, headers=headers)
melonpage = urlopen(request)
# 방법 1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, "html.parser")
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>멜론차트&gt;TOP100&gt;멜론</title>
<meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
<meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
<meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
<meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
<meta content="4022717807957185" property="fb:app_id"/>
<meta content="Melon" property="og:title"/>
<meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
<meta content="음악이 필요한 순간, 멜론" property="og:description"/>
<meta content="http://www.melon.com/chart/" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="멜론" property="og:s

In [ ]:
# 멜론 순위, 노래제목, 가수, 좋아요갯수
# 1위 | 너에게 닿기를 | 10cm | ♡ 85,756  (100위까지 크롤링)

In [272]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [292]:
title_els = soup.select('div.ellipsis rank01 > span')
singer_els = soup.select('div.ellipsis rank02 > checkEllipsis')
len([title.text.strip() for title in title_els]), \
len([singer.text for singer in singer_els])
for idx, (title, singer) in enumerate(zip(title_els, singer_els)):
    print('{}위 | {}-{}'.format(idx+1,
                              titie.text.strip(),
                              singer.text))

In [293]:
# User-Agent 추가
headers = {'user-agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
}
# 방법2
request = Request(url, headers=headers)
melonpage = urlopen(request)
# 방법1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    "html.parser")
#soup

In [294]:
# 멜론 순위, 노래제목, 가수, 좋아요갯수
# 1위 | 너에게 닿기를 | 10cm (100위까지 크롤링)
title_els = soup.select('div.ellipsis.rank01 > span')
singer_els = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
len([title.text.strip() for title in title_els]), \
len([singer.text for singer in singer_els])
for idx, (title, singer) in enumerate(zip(title_els, singer_els)):
    print('{}위 | {} - {}'.format(idx+1,
                                 title.text.strip(),
                                 singer.text))

1위 | 너에게 닿기를 - 10CM
2위 | Never Ending Story - 아이유
3위 | Drowning - WOODZ
4위 | like JENNIE - 제니 (JENNIE)
5위 | 모르시나요(PROD.로코베리) - 조째즈
6위 | THUNDER - 세븐틴 (SEVENTEEN)
7위 | 네모의 꿈 - 아이유
8위 | TOO BAD (feat. Anderson .Paak) - G-DRAGON
9위 | Whiplash - aespa
10위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGON
11위 | 나는 반딧불 - 황가람
12위 | 어제보다 슬픈 오늘 - 우디 (Woody)
13위 | 오늘만 I LOVE YOU - BOYNEXTDOOR
14위 | 빨간 운동화 - 아이유
15위 | REBEL HEART - IVE (아이브)
16위 | HAPPY - DAY6 (데이식스)
17위 | Flower - 오반(OVAN)
18위 | APT. - 로제 (ROSÉ), Bruno Mars
19위 | 10월 4일 - 아이유
20위 | 청춘만화 - 이무진
21위 | HOT - LE SSERAFIM (르세라핌)
22위 | Last Scene (Feat. 원슈타인) - 아이유
23위 | 한 페이지가 될 수 있게 - DAY6 (데이식스)
24위 | MY LOVE(2025) - 이예은, 아샤트리, 전건호
25위 | 그날이 오면 - 투모로우바이투게더
26위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴
27위 | HBD - 세븐틴 (SEVENTEEN)
28위 | 소나기 - 이클립스 (ECLIPSE)
29위 | Welcome to the Show - DAY6 (데이식스)
30위 | 천상연 - 이창섭
31위 | Die With A Smile - Lady Gaga, Bruno Mars
32위 | HANDS UP - MEOVV (미야오)
33위 | Fly Up - RIIZE
34위 | Supernova - aespa
35위 | toxic till the end -

### 5) 네이버 지식인으로 검색(opne API 사용X)
- 특정 keyword를 특정페이지 수(3)만큼 검색한 결과 데이터프레임으로 출력

In [23]:
# 방법 1
from requests import get
from bs4 import BeautifulSoup

keyword = '챗지피티'
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
# print(url)
response = get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [25]:
# 방법 2 
from urllib.request import urlopen
from urllib.parse import quote
# 챗지피티 -> %EC%B1%97%EC%A7%80%ED%94%BC%ED%8B%B0
keyword = quote(keyword)
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
print(url)
response= urlopen(url)
soup = BeautifulSoup(response, 'html.parser')

https://kin.naver.com/search/list.naver?query=%25EC%25B1%2597%25EC%25A7%2580%25ED%2594%25BC%25ED%258B%25B0


In [51]:
# 페이징 포함(keyword 를 pages수 만큼 검색한 결과를 데이터프레임에 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
keyword = '챗지피티'
pages = 3
items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
#     print(url)
# 해당 url의 soup객체 -> 글제목, ; link를 가져와서 items_list에 append
    response = requests.get(url)
    # print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('dt > a')
    # print([item.attrs['href'] for item in items])
    for item in items:
        items_list.append([item.text, item.attrs.get('href')])
#     items_list.append({
#         'title':item.text,
#         'link':items.attrs.get('href')
#     })
# print(len(items_list), '개 리스트')
# print(items_list[:2])
df = pd.DataFrame(items_list)
df

,0,1
0,챗지피티 사주,https://kin.naver.com/qna/detail.naver?d1id=3&...
1,챗지피티 타로,https://kin.naver.com/qna/detail.naver?d1id=3&...
2,챗지피티 타로,https://kin.naver.com/qna/detail.naver?d1id=3&...
3,챗지피티 갸륵체 어떻게 생각하시나요?,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,챗지피티 사용하려면 어떻게 해야 하나요?,https://kin.naver.com/qna/detail.naver?d1id=4&...
5,챗지피티 잡아내는거..,https://kin.naver.com/qna/detail.naver?d1id=4&...
6,챗지피티 개인정보,https://kin.naver.com/qna/detail.naver?d1id=1&...
7,챗지피티 돈 내면서 쓰는거 좋나요?,https://kin.naver.com/qna/detail.naver?d1id=6&...
8,카피킬러 표절 및 챗지피티 검사 83%,https://kin.naver.com/qna/detail.naver?d1id=11...
9,챗지피티 무료 렉 거림,https://kin.naver.com/qna/detail.naver?d1id=1&...


In [53]:
# 페이징 포함(keyword 를 pages수 만큼 검색한 결과를 데이터프레임에 )
import requests
from bs4 import BeautifulSoup
import pandas as pd
keyword = '챗지피티'
pages = 3
items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
#     print(url)
# 해당 url의 soup객체 -> 글제목, ; link를 가져와서 items_list에 append
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dts = soup.find_all('dt')
    for dt in dts:
        item = dt.find('a')
        items_list.append([item.text, item.attrs.get('href')])
df = pd.DataFrame(items_list)
df

,0,1
0,챗지피티 사주,https://kin.naver.com/qna/detail.naver?d1id=3&...
1,챗지피티 타로,https://kin.naver.com/qna/detail.naver?d1id=3&...
2,챗지피티 타로,https://kin.naver.com/qna/detail.naver?d1id=3&...
3,챗지피티 갸륵체 어떻게 생각하시나요?,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,챗지피티 사용하려면 어떻게 해야 하나요?,https://kin.naver.com/qna/detail.naver?d1id=4&...
5,챗지피티 잡아내는거..,https://kin.naver.com/qna/detail.naver?d1id=4&...
6,챗지피티 개인정보,https://kin.naver.com/qna/detail.naver?d1id=1&...
7,챗지피티 돈 내면서 쓰는거 좋나요?,https://kin.naver.com/qna/detail.naver?d1id=6&...
8,카피킬러 표절 및 챗지피티 검사 83%,https://kin.naver.com/qna/detail.naver?d1id=11...
9,챗지피티 무료 렉 거림,https://kin.naver.com/qna/detail.naver?d1id=1&...


In [62]:
# 함수(keyword 를 pages수 만큼 검색한 결과를 데이터프레임에 )
def get_items_data(keyword, pages):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    items_list = [] # 크롤링 한 데이터를 담을 list(2차원 리스트)
    for page in range(1, pages+1):
        url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
        # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        dts = soup.find_all('dt')
        for dt in dts:
            item = dt.find('a')
            items_list.append([item.text, item.attrs.get('href')])
    return pd.DataFrame(items_list, columns=['title', 'link'])

In [63]:
get_items_data('청바지', 3)

,title,link
0,영어로 청바지를 뭐라고하나요?,https://kin.naver.com/qna/detail.naver?d1id=11...
1,"전 남자인데, 누워서 청바지를 입으면...",https://kin.naver.com/qna/detail.naver?d1id=7&...
2,청바지 없는데 면접취소 할까요?,https://kin.naver.com/qna/detail.naver?d1id=6&...
3,고관절 지방이식 후 청바지,https://kin.naver.com/qna/detail.naver?d1id=7&...
4,임영웅 청바지 입은 사진?,https://kin.naver.com/qna/detail.naver?d1id=3&...
5,청바지 세탁,https://kin.naver.com/qna/detail.naver?d1id=8&...
6,미국에서 청바지 가격이 306불입니다...,https://kin.naver.com/qna/detail.naver?d1id=4&...
7,청바지? 면바지도 트레이닝바지인가요?,https://kin.naver.com/qna/detail.naver?d1id=13...
8,핀턱 청바지 수선,https://kin.naver.com/qna/detail.naver?d1id=8&...
9,청바지 이염,https://kin.naver.com/qna/detail.naver?d1id=5&...


### 6) 네이버 지식인으로 검색(opne API 사용◎)
- 특정 keyword를 특정페이지 수(30)만큼 검색한 결과 데이터프레임으로 출력
- .env에 발급받은 키 저장 -> load (pip install python-dotenv)

In [77]:
# 환경변수를 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('client_ID'))
# print(os.getenv('Client_Secret'))

True

In [88]:
# 네이버 검색 API 예제 - 지식인 검색
import os
import sys
import urllib.request
import json
client_id = os.getenv('client_ID')
client_secret = os.getenv('Client_Secret')
encText = urllib.parse.quote("챗지피티")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/kin.xml?query=" + encText # XML 결과

headers = {'X-Naver-Client-Id':client_id,
          'X-Naver-Client-Secret':client_secret}
request = urllib.request.Request(url, headers=headers)
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8')))
    data = json.loads(response_body.decode('utf-8')) # json형태의 str을 딕셔너리
    print(type(data))
    #print(data['items'])
else:
    print("Error Code:" + rescode)

<class 'str'>
<class 'dict'>


In [102]:
# keyword를 display수 만큼 지식in검색 결과를 데이터프레임에(방법 1 크롤링)
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
load_dotenv()
client_id = os.getenv('client_ID')
client_secret = os.getenv('Client_Secret')
keyword = '챗지피티'
cnt = 30
url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
headers = {'X-Naver-Client-Id':client_id,
          'X-Naver-Client-Secret':client_secret}
response = requests.get(url, headers=headers)
# response.status_code
# items = json.loads(response.text)['items'] :  json.loads() - 방법 1, 방법 2
items = response.json()['items'] # 방법1에서만 가능한 함수
items_list = []
for item in items:
    items_list.append([
                        item.get('title').replace('<b>','').replace('</b>',''),  # item[title]
                      item.get('link'),
                       item.get('description')
                      ])
df = pd.DataFrame(items_list, columns=['title','link','description'])
print(df.loc[0, 'link'])

https://kin.naver.com/qna/detail.naver?d1id=3&dirId=31501&docId=484827506&qb=7LGX7KeA7ZS87Yuw&enc=utf8


In [106]:
# 함수 작성
def get_naver_kin(keyword, cnt):
    import requests
    import pandas as pd
    import json
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('client_ID')
    client_secret = os.getenv('Client_Secret')
    url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
    headers = {'X-Naver-Client-Id':client_id,
              'X-Naver-Client-Secret':client_secret}
    response = requests.get(url, headers=headers)
    # items = json.loads(response.text)['items'] :  json.loads() - 방법 1, 방법 2
    items = response.json()['items'] # 방법1에서만 가능한 함수
    items_list = []
    for item in items:
        items_list.append([
                            item.get('title').replace('<b>','').replace('</b>',''),  # item[title]
                          item.get('link'),
                           item.get('description').replace('<b>','').replace('</b>','')
                          ])
    return pd.DataFrame(items_list, columns=['title','link','description'])

In [107]:
get_naver_kin('청바지', 20)

,title,link,description
0,영어로 청바지를 뭐라고하나요?,https://kin.naver.com/qna/detail.naver?d1id=11...,영어로 청바지를 뭐라고하나요? 안녕하세요~ 선생님도 매월 시험보는 목동영어학원 길벗...
1,"전 남자인데, 누워서 청바지를 입으면 고환이 아플 수?",https://kin.naver.com/qna/detail.naver?d1id=7&...,"전 남자인데, 누워서 청바지를 입으면 고환이 아플 수? 안녕하세요. 하이닥-네이버 ..."
2,청바지 없는데 면접취소 할까요?,https://kin.naver.com/qna/detail.naver?d1id=6&...,낼 알바 면접인데 츄리닝만 있고 청바지가 없어요 취소할까요? 아님 청바지는 아니어도...
3,고관절 지방이식 후 청바지,https://kin.naver.com/qna/detail.naver?d1id=7&...,"현재 고관절(골반) 지방이식 한 지 두달 가까이 되어가는데,꽉 끼는 청바지나 레깅스..."
4,임영웅 청바지 입은 사진?,https://kin.naver.com/qna/detail.naver?d1id=3&...,임영웅 사진첩 만들고 있는데요! 청바지 입은 사진 갖고 계신분 공유부탁드려요! 임영...
5,청바지 세탁,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 청바지 구매하고 처음 한번은 드라이크리닝으로 했는데요. 한번 드라이크리닝 했...
6,미국에서 청바지 가격이 306불입니다 관부가세 얼마나오,https://kin.naver.com/qna/detail.naver?d1id=4&...,청바지 디스쿼드 샌스에서 309불에 사려고 합니다 배대지 이용합니다... 청바지 수...
7,청바지? 면바지도 트레이닝바지인가요?,https://kin.naver.com/qna/detail.naver?d1id=13...,제가 내일 체육대회인데여 트레이닝바지가 된다는데 제가 가지고있는바지중에서 카키색에 ...
8,핀턱 청바지 수선,https://kin.naver.com/qna/detail.naver?d1id=8&...,핀턱있는 청바지 허리 줄일 수 있나용 네! 핀턱 청바지도 허리 수선이 가능합니다. ...
9,청바지 이염,https://kin.naver.com/qna/detail.naver?d1id=5&...,... 아니면 진한 청바지는 다 이염이 된다고 봐야하나요ㅠ 진한 청바지 모두에서 이...


### quiz) 네이버 opne API를 이용해서 청바지 이미지 100건 데이터를 csv파일로 저장
 - 데이터 : 제목, 이미지링크, 썸네일링크, 높이, 폭

In [177]:

import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
word = '슬렉스'
word = urllib.parse.quote(word)
client_id = os.getenv('client_ID')
client_secret = os.getenv('Client_Secret')
url = f'https://openapi.naver.com/v1/search/image.json?query={word}&display=100'
headers = {'X-Naver-Client-Id':client_id,
            'X-Naver-Client-Secret':client_secret}
response = requests.get(url, headers=headers)
items = json.loads(response.text)['items'] 
items_list = []
# for item in items:
for idx, item in enumerate(items):
    title = item.get('title')
    link = item.get('link')
    thumbnail = item.get('thumbnail')
    sizeheight = item.get('sizeheight')
    sizewidth = item.get('sizewidth')
    items_list.append({
        'no': idx+1,
        'title': title,
        'link' : link,
        'thumbnail' : thumbnail,
        'sizeheight':sizeheight,
        'sizewidth' :sizewidth
                    })
#                         item.get('title'),  # item[title]
#                        item.get('link'),
#                        item.get('thumbnail'),
#                        item.get('sizeheight'),
#                        item.get('sizewidth')
#                       ])
# df = pd.DataFrame(items_list, columns=['title','link','thumbnail','sizeheight','sizewidth'])
df = pd.DataFrame(items_list)
df

,no,title,link,thumbnail,sizeheight,sizewidth
0,1,남자 9부 스판 정장바지 청바지 밴딩슬랙스 밴딩팬츠 면바지 슬렉스 슬랙스DJ-SLA...,http://shop1.phinf.naver.net/20240403_110/1712...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
1,2,1 10부 S 프리미엄 밴딩 슬렉스 P 3008,http://shopping.phinf.naver.net/main_4295129/4...,https://search.pstatic.net/common/?type=b150&s...,700,600
2,3,베이직바이블 남자 빅사이즈 슬랙스 9부10부 빅사이즈 스판 바지(5color) - ...,https://thumbnail7.coupangcdn.com/thumbnails/r...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
3,4,와이드 모르 슬렉스 : Butee,http://shop1.phinf.naver.net/20240729_44/17222...,https://search.pstatic.net/common/?type=b150&s...,1500,1000
4,5,(1+1) 베이직 남성 남자 여름 기획 슬랙스 (3color) 가을 슬렉스 간절기 ...,http://shop1.phinf.naver.net/20210504_232/1620...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
...,...,...,...,...,...,...
95,96,프리미엄 은장장식 썸머 롱다리핏 스판 슬렉스 와이드 밴딩 통바지 하이웨스트 페리모 ...,http://shop1.phinf.naver.net/20230616_58/16868...,https://search.pstatic.net/common/?type=b150&s...,825,860
96,97,울라이크 스트레치 슬렉스 2컬러 NBB4IPP1953243699,http://shopping.phinf.naver.net/main_5257979/5...,https://search.pstatic.net/common/?type=b150&s...,500,500
97,98,울트라 텐션 슬렉스 : Butee,http://shop1.phinf.naver.net/20210213_119/1613...,https://search.pstatic.net/common/?type=b150&s...,1500,1000
98,99,아이스타일24 [인터메조(AK 남성의류)][인터메조] 슬림라인 스판 슬렉스 65P2...,http://photo.akmall.com/image4/goods/72/19/99/...,https://search.pstatic.net/sunny/?type=b150&sr...,500,500


In [135]:
df.loc[3, 'link']

'https://thumbnail8.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/images/2017/09/07/17/1/837feb8c-041e-4921-bd39-174fec765a2a.jpg'

In [136]:
df[df['link'].str.find('?')!=-1]

,no,title,link,thumbnail,sizeheight,sizewidth


In [143]:
df[~df['link'].str.endswith('jpg')].loc[5,'link']

'http://shop1.phinf.naver.net/20250503_295/1746250611381Vinxl_JPEG/21192829047529977_2039968073.jpeg'

In [166]:
def save_img(attr, idx, query, url):
    '{attr}_{idx}_{query}.{확장자} 이미지 저장'
    file_extension = url.split('.')[-1]
    quote_index = file_extension.find('?')
    
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
#     print(file_extension)
    img = requests.get(url).content
    open(f'ch14_image/{attr}_{idx}_{query}.{file_extension}', 'wb').write(img)
        

In [186]:
save_img('메인', 1, '청바지',df.loc[1, 'link'])

In [187]:
# word image 100개 검색한 데이터는csv파일로, 이미지는 ch14_image폴더에 저장되는 함수
def get_naver_save_image(word):
    import requests
    import pandas as pd
    import json
    from dotenv import load_dotenv
    import os
    word = urllib.parse.quote(word)
    client_id = os.getenv('client_ID')
    client_secret = os.getenv('Client_Secret')
    url = f'https://openapi.naver.com/v1/search/image.json?query={word}&display=100'
    headers = {'X-Naver-Client-Id':client_id,
                'X-Naver-Client-Secret':client_secret}
    response = requests.get(url, headers=headers)
    items = json.loads(response.text)['items'] 
    items_list = []
    for idx, item in enumerate(items):
        title = item.get('title')
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        sizeheight = item.get('sizeheight')
        sizewidth = item.get('sizewidth')
        items_list.append({
            'no': idx+1,
            'title': title,
            'link' : link,
            'thumbnail' : thumbnail,
            'sizeheight':sizeheight,
            'sizewidth' :sizewidth
                        })
        save_img('메인', idx+1, word, link)
        save_img('썸네일', idx+1, word, thumbnail)
        # 20% 40% 60% 80% 지점 message 출력
        if idx/20 == round(idx/20):
            print(f'- - → {idx}% 진행중입니다 ← - - ')
    df = pd.DataFrame(items_list)
    df.to_csv(f'ch14_image/{word}.csv', index=False )
    print('- - - 이미지 및 csv 파일 저장 완료 - - -')

In [188]:
get_naver_save_image('청바지')

- - → 0% 진행중입니다 ← - - 
- - → 20% 진행중입니다 ← - - 
- - → 40% 진행중입니다 ← - - 
- - → 60% 진행중입니다 ← - - 
- - → 80% 진행중입니다 ← - - 
- - - 이미지 및 csv 파일 저장 완료 - - -


## 2.3 xml 파일
- 기상예측 RSS 활용
- https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml

In [189]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


In [203]:
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml'
response = urlopen(url)
#  response.status # response.getcode()
soup = BeautifulSoup(response, 'xml') # pip install lxml
locals = soup.find_all('local_ta')
# locals[1]
items_list = []
for local in locals:
    local_ta_name = local.select_one('local_ta_name').text
    week1_normalYear = local.select_one('week1_local_ta_normalYear').text
    week1_similarRangen = local.select_one('week1_local_ta_similarRange').text
    week1_min = local.select_one('week1_local_ta_minVal').text
    week1_max = local.select_one('week1_local_ta_maxVal').text
    week2_normalYear = local.select_one('week2_local_ta_normalYear').text
    week2_similarRangen = local.select_one('week2_local_ta_similarRange').text
    week2_min = local.select_one('week2_local_ta_minVal').text
    week2_max = local.select_one('week2_local_ta_maxVal').text
    items_list.append({
        '지역':local_ta_name,
        '1주평년기온':week1_normalYear,
        '1주범위':week1_similarRangen,
        '1주최저':week1_min,
        '1주최고':week1_max,
        '2주평년기온':week2_normalYear,
        '2주범위':week2_similarRangen,
        '2주최저':week2_min,
        '2주최고':week2_max
    })
df = pd.DataFrame(items_list)
df

,지역,1주평년기온,1주범위,1주최저,1주최고,2주평년기온,2주범위,2주최저,2주최고
0,"전국(제주도,북한제외)",21.1,20.6~21.6,10,60,21.9,21.5~22.3,20,40
1,서울ㆍ인천ㆍ경기도,21.6,21.1~22.1,10,60,22.4,22.0~22.8,20,30
2,강원도 영서,20.3,19.7~20.9,10,60,21.3,20.7~21.9,20,30
3,강원도 영동,19.1,18.2~20.0,10,60,20.1,19.4~20.8,20,30
4,대전ㆍ세종ㆍ충청남도,21.5,21.0~22.0,10,60,22.3,21.9~22.7,20,40
5,충청북도,21.3,20.7~21.9,10,60,22.2,21.8~22.6,20,40
6,광주ㆍ전라남도,21.3,20.9~21.7,10,60,22.0,21.7~22.3,20,40
7,전북자치도,21.3,20.9~21.7,10,60,22.1,21.7~22.5,20,40
8,부산ㆍ울산ㆍ경상남도,21.3,20.8~21.8,10,60,22.1,21.7~22.5,20,40
9,대구ㆍ경상북도,20.9,20.3~21.5,10,60,21.9,21.3~22.5,20,40


### 1) 버스위치정보 조회(OpenAPI활용)
- data.go.kr
    * 서울특별시_노선정보조회 서비스(busRoteId, stationNm-정류소명, station-정류소id)
    * 서울특별시_버스위치정보조회 서비스(busRouteId -> 버스위치)

In [ ]:
# step1 : 버스번호 -> busRouteId
# 노선정보조회 서비스 3번 getBusRouteList 서비스

In [1]:
from dotenv import load_dotenv
import os
load_dotenv () #dotenv_path='.env'
# print('key = ', os.getenv('key'))
# print('skey = ', os.getenv('skey'))

True

In [8]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve # url 경로파일을 내pc에 저장
from urllib.parse import quote
#busNum =  quote('강동01')
busNum = '3214'    
key = os.getenv('key')  # 일반 인증키(Encoding)


url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
# print(url)
# response = BeautifulSoup(response.text, 'xml')
saveilename = 'data/ch14_busInfo.xml'
urlretrieve(url1, saveilename) # url 경로 파일을 savefilename으로 저장
xml = open(saveilename, 'r', encoding='utf-8').read()
soup = BeautifulSoup(xml, 'xml')
for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break
print('busRouteId : ', busRouteId)

busRouteId :  100100213


In [ ]:
# step2 : busRouteId로 버스 정류장 목록(정류장이름과 정류장id)
# 노선정도조회 서비스  4번 : getStaionsByRouteList

In [15]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId }'
print(url2)
response = requests.get(url2)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.find_all('itemList')
print(busNum, '번 정류장 갯수 : ', len(itemLists))
bus_station = []
for itemList in itemLists:
    # 정류장이름, 정류장id, 경도 , 위도
    stationNm = itemList.find('stationNm').text
    station = itemList.find('station').text
    gpsX = itemList.find('gpsX').text
    gpsY = itemList.find('gpsY').text
    bus_station.append([stationNm, station, gpsX, gpsY])
df = pd.DataFrame(bus_station, columns=['정류소명','id','경도','위도'])
df

http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey=kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D&busRouteId=100100213
3214 번 정류장 갯수 :  57


,정류소명,id,경도,위도
0,남한산성입구,123000337,127.157278,37.49096
1,송파구립마천청소년센터,123000200,127.15679,37.491832
2,송파보훈회관.금호어울림2차,123000201,127.1558084636,37.4939681019
3,마천동금호어울림1차아파트,123000203,127.154198,37.497198
4,남천초교.마천동우방아파트앞,123000206,127.1519312753,37.4999186245
5,청암노인요양원,123000208,127.150052,37.501079
6,성내천공원,123000685,127.147632,37.50072
7,마천사거리,123000209,127.1466290856,37.4985293494
8,마천2동윤진빌딩.신동아아파트,123000212,127.1442444466,37.4996759516
9,거여초등학교,123000164,127.1402155083,37.502548108


In [35]:
# step3 : busRouteId로 버스 위치 정보 조회 2번 : (2) getBusPosByRtidList 
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey={key}&busRouteId={busRouteId}'
print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
bus_position = []
itemLists = soup.select('itemList')
print('운행중인 버스 수 : ', len(itemLists))
for itemList in soup.select('itemList'):
    # 차량번호, 혼잡도, 경도, 위도, 최종정류장id, 다음정류장id, 도착소요시간
    plainNo = itemList.select_one('plainNo').text
    busType = itemList.select_one('busType').text
    busType = '일반버스' if busType=='0'\
                     else '저상버스' if busType=='1'\
                     else '굴절버스' 
    congetion = itemList.select_one('congetion').text
    congetion = '여유' if congetion == '3' \
                        else '보통' if congetion=='4' \
                        else '혼잡' if congetion=='5' \
                        else '매우혼잡'
    gpsX = itemList.select_one('gpsX').text
    gpsY = itemList.select_one('gpsY').text
    lastStnId = itemList.select_one('lastStnId').text
    nextStId = itemList.select_one('nextStId').text
    nextStTm = itemList.select_one('nextStTm').text
    bus_position.append({
                         '차량번호' : plainNo,
                         '차량유형' : busType,
                         '혼잡도' : congetion,
                         '경도' : gpsX,
                         '위도' : gpsY,
                         '최종정류소ID' : lastStnId,
                         '다음정류소ID' : nextStId,
                         '도착소요시간' : nextStTm
    })
    
df_pos = pd.DataFrame(bus_position)
df_pos

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=kWiLqR5an3qgHfkycjPmjxAie5mGGzW0LuPktrSNA3kqpsmWfsLSW9TxR9k0khrZgJLecVcO88n5LsStz845eg%3D%3D&busRouteId=100100213
운행중인 버스 수 :  8


,차량번호,차량유형,혼잡도,경도,위도,최종정류소ID,다음정류소ID,도착소요시간
0,서울74사4936,저상버스,여유,127.146566,37.498129,123000209,123000173,501
1,서울71사1933,저상버스,여유,127.12776,37.511216,123000158,124000136,498
2,서울74사4946,저상버스,여유,127.125847,37.537929,124000453,124000026,1893
3,서울74사1244,일반버스,여유,127.099719,37.537964,123000033,124000026,1365
4,서울70사9621,저상버스,여유,127.107275,37.530638,104000064,124000026,885
5,서울74사4943,일반버스,여유,127.137493,37.531296,124000133,124000135,135
6,서울74사9056,저상버스,여유,127.13124,37.506019,123000172,123000210,521
7,서울74사1250,일반버스,여유,127.156381,37.492188,123000199,123000210,120


In [34]:
# id가 123000203인 정류소 명
df.loc[df['id'] == '123000203', '정류소명'].iloc[0]

'마천동금호어울림1차아파트'

In [38]:
def station_name(row):
    row['최종정류소명'] = df.loc[df['id'] == row['최종정류소ID'], '정류소명'].iloc[0]
    row['다음정류소명'] = df.loc[df['id'] == row['다음정류소ID'], '정류소명'].iloc[0]
    return row

In [39]:
df_pos = df_pos.apply(station_name, axis=1)
df_pos

,차량번호,차량유형,혼잡도,경도,위도,최종정류소ID,다음정류소ID,도착소요시간,최종정류소명,다음정류소명
0,서울74사4936,저상버스,여유,127.146566,37.498129,123000209,123000173,501,마천사거리,오금공원사거리
1,서울71사1933,저상버스,여유,127.12776,37.511216,123000158,124000136,498,방이역,둔촌동역2번출구
2,서울74사4946,저상버스,여유,127.125847,37.537929,124000453,124000026,1893,천호역,강동역.강풀만화거리입구
3,서울74사1244,일반버스,여유,127.099719,37.537964,123000033,124000026,1365,올림픽대교남단,강동역.강풀만화거리입구
4,서울70사9621,저상버스,여유,127.107275,37.530638,104000064,124000026,885,현대아파트앞,강동역.강풀만화거리입구
5,서울74사4943,일반버스,여유,127.137493,37.531296,124000133,124000135,135,강동리바트,둔촌동역4번출구
6,서울74사9056,저상버스,여유,127.13124,37.506019,123000172,123000210,521,오금공원사거리,마천사거리
7,서울74사1250,일반버스,여유,127.156381,37.492188,123000199,123000210,120,송파구립마천청소년센터,마천사거리


In [41]:
# 컬럼명에 id가 포함된 컬럼 제거
drop_true = df_pos.columns.str.contains('ID')
drop_column_name = df_pos.columns[drop_true]
df_pos.drop(drop_column_name, axis=1)

,차량번호,차량유형,혼잡도,경도,위도,도착소요시간,최종정류소명,다음정류소명
0,서울74사4936,저상버스,여유,127.146566,37.498129,501,마천사거리,오금공원사거리
1,서울71사1933,저상버스,여유,127.12776,37.511216,498,방이역,둔촌동역2번출구
2,서울74사4946,저상버스,여유,127.125847,37.537929,1893,천호역,강동역.강풀만화거리입구
3,서울74사1244,일반버스,여유,127.099719,37.537964,1365,올림픽대교남단,강동역.강풀만화거리입구
4,서울70사9621,저상버스,여유,127.107275,37.530638,885,현대아파트앞,강동역.강풀만화거리입구
5,서울74사4943,일반버스,여유,127.137493,37.531296,135,강동리바트,둔촌동역4번출구
6,서울74사9056,저상버스,여유,127.13124,37.506019,521,오금공원사거리,마천사거리
7,서울74사1250,일반버스,여유,127.156381,37.492188,120,송파구립마천청소년센터,마천사거리


# 3. 연습문제
- YES24 베스트셀러 정보를 ch14_yes24.csv(ch14_yes24.txt)로 출력
- 순위, 책이름, 저자 및 출판사, 가격 (1위~48위) => 리스트 =>  데이터프레임 => .csv
-                                                 => .txt

In [247]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
bestseller_list = []
pages = 2
url = 'https://www.yes24.com/product/category/bestseller?pageNumber={pages}'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
for page in range(1, pages+1):
        ranking = soup.select('div.img_upper > em.ico')
        rank = [rank.text for rank in ranking]
        books = soup.select('div.info_name > a.gd_name')
        book = [book.text for book in books]
        names = soup.select('div.info_pubGrp > span.info_auth> a')
        name = [name.text for name in names]
        companys  = soup.select('div.info_pubGrp > span.info_pub> a')
        company = [company.text for company in companys]
        prices  = soup.select('strong.txt_num > em.yes_b')
        price = [price.text for price in prices]
        for rank,book,name,company,price in zip(ranking,books,names,companys,prices):
            bestseller_list.append({
                '순위':rank,
                '제목':book,
                '저자':name,
                '출판사':company,
                '가격':price
            })
df = pd.DataFrame(bestseller_list, 
                  columns=['순위', '제목', '저자', '출판사', '가격'])
df.to_csv('data/ch14_yes24.csv', index=False)

In [ ]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    # print(url)
    response = urlopen(url)
    #print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks = [int(ranks_el.text) for ranks_el in ranks_els]
    titles_els=soup.select("div.item_info > div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    writers_els = soup.select("span.authPub.info_auth")
    # writers_els = soup.find_all('span', class_='info_auth')
    writers = [writers_el.text.strip() for writers_el in writers_els]
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, writer, publisher, price in zip(ranks, titles, writers, publishers, prices):
            # print("{},{},{},{},{}".format(rank, title, writer, publisher, price))
            bestseller_list.append({
                'rank':rank, 
                'title':title,
                'writer':writer,
                'publisher':publisher,
                'price':price})
            f.write(f'{rank}, {title}, {writer}, {publisher}, {price}\n')
df = pd.DataFrame(bestseller_list)
df.to_csv('data/ch14_yes24.csv', index=False)